In [1]:
import pandas as pd
import time
import random
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

In [14]:
# Input
INPUT_FILE = 'OUTPUT\metro_intersecting_parcels_HCAD_20250505_220658.csv'

In [15]:
OUTPUT_FILE = 'OUTPUT\hcad_parcel_details.csv'

In [4]:
def get_parcel_data(hcad_num):
    driver.get("https://hcad.org/property-search/property-search")
    time.sleep(random.uniform(3, 5))

    try:
        search_box = driver.find_element(By.ID, "search-text")
        search_box.clear()
        search_box.send_keys(hcad_num)
        search_box.submit()
        time.sleep(random.uniform(4, 6))

        return {
            "HCAD_NUM": hcad_num,
            "Owner": driver.find_element(By.CSS_SELECTOR, ".owner-name").text,
            "Address": driver.find_element(By.CSS_SELECTOR, ".address").text,
            "Account Type": driver.find_element(By.CSS_SELECTOR, ".account-type").text,
            "Total Appraised Value": driver.find_element(By.XPATH, "//div[contains(text(),'Total Appraised Value')]/following-sibling::div").text
        }
    except Exception as e:
        print(f"[ERROR] {hcad_num}: {e}")
        return {"HCAD_NUM": hcad_num, "Owner": None, "Address": None, "Account Type": None, "Total Appraised Value": None}



In [16]:
# Load HCAD_NUMs
df = pd.read_csv(INPUT_FILE)
hcad_nums = df['HCAD_NUM'].astype(str).tolist()

In [17]:
# Resume from previous progress
if os.path.exists(OUTPUT_FILE):
    scraped = pd.read_csv(OUTPUT_FILE)
    done = set(scraped['HCAD_NUM'].astype(str))
else:
    scraped = pd.DataFrame()
    done = set()

remaining = [num for num in hcad_nums if num not in done]


In [18]:
# Set up Selenium
chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(options=chrome_options)

In [19]:
# Scrape in batches
batch = []
for idx, num in enumerate(remaining, 1):
    record = get_parcel_data(num)
    batch.append(record)

    if idx % 50 == 0 or idx == len(remaining):
        pd.DataFrame(batch).to_csv(OUTPUT_FILE, mode='a', header=not os.path.exists(OUTPUT_FILE), index=False)
        print(f"[SAVED] {idx} records processed.")
        batch.clear()
        time.sleep(random.uniform(5, 10))

driver.quit()

[ERROR] 1155930140016: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="search-text"]"}
  (Session info: chrome=135.0.7049.115); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF692D0EFA5+77893]
	GetHandleVerifier [0x00007FF692D0F000+77984]
	(No symbol) [0x00007FF692AD91BA]
	(No symbol) [0x00007FF692B2F16D]
	(No symbol) [0x00007FF692B2F41C]
	(No symbol) [0x00007FF692B82237]
	(No symbol) [0x00007FF692B5716F]
	(No symbol) [0x00007FF692B7F07F]
	(No symbol) [0x00007FF692B56F03]
	(No symbol) [0x00007FF692B20328]
	(No symbol) [0x00007FF692B21093]
	GetHandleVerifier [0x00007FF692FC7B6D+2931725]
	GetHandleVerifier [0x00007FF692FC2132+2908626]
	GetHandleVerifier [0x00007FF692FE00F3+3031443]
	GetHandleVerifier [0x00007FF692D291EA+184970]
	GetHandleVerifier [0x00007FF692D3086F+215311]
	GetHandleVerifier [0x00007FF692D

KeyboardInterrupt: 